# This is a ViT B/16 code on performance of it

In [ ]:
#mount the files and access first
from google.colab import drive
drive.mount('/content/drive')
# Then your model is at /content/drive/MyDrive/EC523_data/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# =========================
# ViT-B/16 Fine-tuning Script (Colab + Google Drive)
# =========================

# -------------------------
# 0. Install dependencies
# -------------------------
!pip install transformers safetensors

# -------------------------
# 1. Mount Google Drive
# -------------------------
from google.colab import drive
drive.mount('/content/drive')

# Update this path to the folder where your EC523_data is located in Drive
model_dir = "/content/drive/MyDrive/EC523_data"

train_dir = f"{model_dir}/formal data/Composite data/Training"
val_dir   = f"{model_dir}/formal data/Composite data/Validation"
test_dir  = f"{model_dir}/formal data/Composite data/Testing"

train_csv = f"{train_dir}/Training_labels.csv"
val_csv   = f"{val_dir}/Validation_labels.csv"
test_csv  = f"{test_dir}/Testing_labels.csv"

model_file = "model.safetensors"  # pretrained ViT-B/16

# -------------------------
# 2. Imports
# -------------------------
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import ast
import matplotlib.pyplot as plt
from transformers import ViTConfig, ViTForImageClassification
import safetensors.torch

# -------------------------
# 3. PyTorch preprocessing
# -------------------------
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

# -------------------------
# 4. Dataset class
# -------------------------
class CustomImageDataset(Dataset):
    def __init__(self, images_dir, csv_file, transform=None):
        self.images_dir = images_dir
        self.df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['image_name']
        img_path = os.path.join(self.images_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        # Convert string representation of one_hot to tensor
        label = torch.tensor(ast.literal_eval(self.df.iloc[idx]['one_hot']), dtype=torch.float)
        return image, label

# -------------------------
# 5. Load datasets & dataloaders
# -------------------------
batch_size = 16

train_dataset = CustomImageDataset(train_dir, train_csv, transform=preprocess)
val_dataset   = CustomImageDataset(val_dir, val_csv, transform=preprocess)
test_dataset  = CustomImageDataset(test_dir, test_csv, transform=preprocess)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# -------------------------
# 6. Load ViT-B/16 model
# -------------------------
# Determine number of classes from one-hot labels
num_classes = len(ast.literal_eval(pd.read_csv(train_csv)['one_hot'][0]))

config = ViTConfig(
    image_size=224,
    patch_size=16,
    num_channels=3,
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
    num_labels=num_classes
)

model = ViTForImageClassification(config)

# Load pretrained weights
state_dict_path = os.path.join(model_dir, model_file)
state_dict = safetensors.torch.load_file(state_dict_path)
model.load_state_dict(state_dict, strict=False)  # strict=False allows classifier replacement

# Replace classifier
model.classifier = nn.Linear(model.classifier.in_features, num_classes)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# -------------------------
# 7. Loss & optimizer
# -------------------------
criterion = nn.BCEWithLogitsLoss()  # for one-hot multi-class labels
optimizer = optim.Adam(model.parameters(), lr=3e-5)

# -------------------------
# 8. Training loop
# -------------------------
num_epochs = 5
train_losses, val_losses = [], []

for epoch in range(num_epochs):
    # ---- Training ----
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_train_loss)

    # ---- Validation ----
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
    epoch_val_loss = val_loss / len(val_loader.dataset)
    val_losses.append(epoch_val_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {epoch_train_loss:.4f} | Val Loss: {epoch_val_loss:.4f}")

# -------------------------
# 9. Plot training & validation loss
# -------------------------
plt.figure(figsize=(8,6))
plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs+1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()
plt.show()

# -------------------------
# 10. Testing
# -------------------------
model.eval()
test_loss = 0.0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)
test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/EC523_data/formal data/Composite data/Training/Training_labels.csv'

In [ ]:
!ls "/content/drive/MyDrive/EC523_data"

ls: cannot access '/content/drive/MyDrive/EC523_data': No such file or directory
